In [ ]:
import os
import sys

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

from datacube.utils.aws import configure_s3_access
configure_s3_access(requester_pays=True)

import datacube
dc = datacube.Datacube()

In [ ]:
product = 's1_rtc_card4l'

In [ ]:
meas_info = dc.list_measurements()

In [ ]:
nodata_vals = meas_info.loc[product]['nodata']

In [ ]:
nodata_vals

In [ ]:
lat = (36.5, 37)
lon = (-76.5, -76)

In [ ]:
# !aws s3 --request-payer=requester ls sh.s1-card4l.eu-central-1.nasa/order_2021-03-17T15:13:58Z/s1_rtc/N36W077/2017/01/05/017ED1/s1_rtc_01BD07_N36W083_2017_05_25_VV.tif

In [ ]:
# Try indexing N36W077/2017/01/05/017ED1 and 
#              N36W083/2017/05/25/01BD07 (?)

In [ ]:
lat, lon

In [ ]:
ds = dc.load(product=product, lat=lat, lon=lon,
               time=('2017-01-01', '2017-01-31'), 
               measurements=['vv'], group_by='solar_day',
               dask_chunks=dict(time=1, latitude=2000, longitude=2000)
            )

In [ ]:
ds

In [ ]:
ds = ds.compute()

In [ ]:
inf_mask = np.isfinite(ds)

In [ ]:
nan_mask = ~np.isnan(ds)

In [ ]:
no_data_mask = xr.merge([ds[data_var] != nodata_vals[data_var] for data_var in ds]).to_array().all('variable')

In [ ]:
ds = ds.where(inf_mask & nan_mask & no_data_mask)

In [ ]:
min_ = np.nanpercentile(ds.vv, 5)
max_ = np.nanpercentile(ds.vv, 95)

ds = ds.where((min_<ds) & (ds<max_))

**Show the mean of the data**

In [ ]:
ds['vv'].mean('time').plot.imshow()

**Show each day as an image**

In [ ]:
# for time in ds.time:
#     ds['vv'].sel(time=time).plot.imshow()
#     plt.show()